# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
#Необходимые библиотеки
import ast
from datetime import datetime, timedelta 

# Иструмент для кодирования
from sklearn.preprocessing import LabelEncoder
import re


 Restaurant_id — идентификационный номер ресторана / сети ресторанов;
 City — город, в котором находится ресторан;
 Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
 Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
 Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
 Price Range — диапазон цен в ресторане;
 Number of Reviews — количество отзывов о ресторане;
 Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
 URL_TA — URL страницы ресторана на TripAdvisor;
 ID_TA — идентификатор ресторана в базе данных TripAdvisor.


In [2]:
df= pd.read_csv('main_task.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [3]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)  
y = df['Rating']  
      
    # Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
      
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
    # Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    

In [4]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

Количество столбцов - 10 
В числовом формате:Ranking, Rating, Number of Reviews.

Содержат пропущенные (None) значения: Cuisine Style, Price Range, Number of Reviews 




In [5]:
#переименуем столбцы
df.rename(columns={
    'Restaurant_id': 'restaurant_id',
    'City': 'city',
    'Cuisine Style': 'cuisine_style',
    'Ranking': 'ranking',
    'Rating': 'rating',
    'Price Range': 'price_range',
    'Number of Reviews': 'number_of_reviews',
    'Reviews': 'reviews',
    'URL_TA': 'url_ta',
    'ID_TA': 'id_ta'
    }, inplace=True)


Заполним медианой пропуски в количестве отзывов:

In [6]:
df['number_of_reviews_NaN']=pd.isna(df['number_of_reviews'])
df['number_of_reviews_NaN'].value_counts()


False    37457
True      2543
Name: number_of_reviews_NaN, dtype: int64

In [7]:
df['number_of_reviews']=df['number_of_reviews'].fillna(df['number_of_reviews'].median())

In [8]:
# Считаем количество городов в наборе данных:
len(df.city.value_counts())

31

price_range
Диапазон цен в ресторане имеет три варианта значений:

    $ - низкий
    $$ - $$$ - средний
    $$$$ - высокий


In [9]:
df[df['price_range']=='$$ - $$$'].count()


restaurant_id            18412
city                     18412
cuisine_style            18412
ranking                  18412
rating                   18412
price_range              18412
number_of_reviews        18412
reviews                  18412
url_ta                   18412
id_ta                    18412
number_of_reviews_NaN    18412
dtype: int64

In [10]:

def price(s):
    if s == '$':
        return 1
    elif s == '$$ - $$$':
        return 2
    elif s == '$$$$':
        return 3
    else:
        return 0

df['price_range']= df['price_range'].apply(lambda x: price(x))
df.head(15)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,number_of_reviews_NaN
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,False
1,id_1535,Stockholm,NaN,1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,False
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,False
3,id_3456,Berlin,NaN,3458.0,5.0,0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,False
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,False
5,id_1418,Oporto,NaN,1419.0,3.0,0,2.0,"[['There are better 3 star hotel bars', 'Amazi...",/Restaurant_Review-g189180-d12503536-Reviews-D...,d12503536,False
6,id_1720,Milan,"['Italian', 'Pizza']",1722.0,4.0,1,50.0,"[['Excellent simple local eatery.', 'Excellent...",/Restaurant_Review-g187849-d5808504-Reviews-Pi...,d5808504,False
7,id_825,Bratislava,['Italian'],826.0,3.0,0,9.0,"[['Wasting of money', 'excellent cuisine'], ['...",/Restaurant_Review-g274924-d3199765-Reviews-Ri...,d3199765,False
8,id_2690,Vienna,NaN,2692.0,4.0,0,33.0,"[[], []]",/Restaurant_Review-g190454-d12845029-Reviews-G...,d12845029,True
9,id_4209,Rome,"['Italian', 'Pizza', 'Fast Food']",4210.0,4.0,1,55.0,"[['Clean efficient staff', 'Nice little pizza ...",/Restaurant_Review-g187791-d8020681-Reviews-Qu...,d8020681,False


In [11]:
df.price_range.sample(5)

6467     2
33500    1
34254    2
20689    1
4251     0
Name: price_range, dtype: int64

for index,i in enumerate(data['Price Range']):
    data['Price Range'][index]=price_range[i]
data['Price Range']



cuisine_style

In [12]:
df.cuisine_style.sample(5)



12504                         ['Italian', 'Mediterranean']
16276                              ['Vegetarian Friendly']
23991                            ['American', 'Fast Food']
15605    ['Italian', 'Pizza', 'Mediterranean', 'Egyptian']
37692                                                  NaN
Name: cuisine_style, dtype: object

In [13]:
#Заполнение пустых значений
df.cuisine_style.fillna('[nan]', inplace=True)

In [14]:
# конвертируем строки в списки

df.cuisine_style = df.cuisine_style.apply(lambda x: x if pd.isna(x)
                              else x[2:-2])

df.cuisine_style = df.cuisine_style.apply(lambda x: x if pd.isna(x)
                              else x.split("', '"))

In [15]:
df['number_of_cuisines'] = df.cuisine_style.apply(lambda x: len(x))

In [16]:

cuisine_values = df.cuisine_style.explode().value_counts()
cuisine_list = [x for x in cuisine_values.index]
cuisine_list

['Vegetarian Friendly',
 'European',
 'a',
 'Mediterranean',
 'Italian',
 'Vegan Options',
 'Gluten Free Options',
 'Bar',
 'French',
 'Asian',
 'Pizza',
 'Spanish',
 'Pub',
 'Cafe',
 'Fast Food',
 'British',
 'International',
 'Seafood',
 'Japanese',
 'Central European',
 'American',
 'Sushi',
 'Chinese',
 'Portuguese',
 'Indian',
 'Middle Eastern',
 'Thai',
 'Wine Bar',
 'German',
 'Healthy',
 'Greek',
 'Halal',
 'Czech',
 'Fusion',
 'Steakhouse',
 'Barbecue',
 'Contemporary',
 'Vietnamese',
 'Eastern European',
 'Soups',
 'Grill',
 'Gastropub',
 'Mexican',
 'Turkish',
 'Delicatessen',
 'Austrian',
 'South American',
 'Polish',
 'Hungarian',
 'Scandinavian',
 'Lebanese',
 'Latin',
 'Diner',
 'Dutch',
 'Irish',
 'Belgian',
 'Street Food',
 'Brew Pub',
 'Swiss',
 'Danish',
 'Swedish',
 'Argentinean',
 'Korean',
 'Scottish',
 'African',
 'Moroccan',
 'Central American',
 'Brazilian',
 'Pakistani',
 'Peruvian',
 'Caribbean',
 'Nepali',
 'Balti',
 'Bangladeshi',
 'Norwegian',
 'Israeli',


In [17]:
group_city_cuisine = df.groupby(['city'])['cuisine_style'].sum().reset_index()
 

In [18]:
#df.groupby('Pclass')['Age'].apply(lambda x:x.fillna(x.mean()))

In [19]:
import collections

In [20]:
c = collections.Counter()

for cuisine_style in group_city_cuisine.cuisine_style:
    for x in cuisine_style:
        c[x] += 1

print(c.most_common(10))
print()
print(c.most_common()[-10:])

[('Vegetarian Friendly', 11189), ('European', 10060), ('a', 9283), ('Mediterranean', 6277), ('Italian', 5964), ('Vegan Options', 4486), ('Gluten Free Options', 4113), ('Bar', 3297), ('French', 3190), ('Asian', 3011)]

[('Ecuadorean', 4), ('Polynesian', 2), ('Azerbaijani', 2), ('Welsh', 2), ('Fujian', 2), ('Yunnan', 1), ('Latvian', 1), ('Burmese', 1), ('Salvadoran', 1), ('Xinjiang', 1)]


In [21]:
df.isnull().sum()

restaurant_id            0
city                     0
cuisine_style            0
ranking                  0
rating                   0
price_range              0
number_of_reviews        0
reviews                  0
url_ta                   0
id_ta                    0
number_of_reviews_NaN    0
number_of_cuisines       0
dtype: int64

In [22]:
def processing_reviews(df):
    """
    Функция разбирает строку с комментарием и датой 
    и помещает комментарии и даты в 4 новые колонки 
    2 для комментариев и 2 для дат
    
    """
    
    
    reviews_fb_1 = []
    reviews_fb_2 = []
    reviews_date_1 = []
    reviews_date_2 = []
    
    for index, row in df.iterrows():
        str_review = ast.literal_eval(str(row['reviews'].replace('nan','0')))   # Разбили строку с комментарием на пополам (комменты/даты)
        
        if str_review == [[], []]:
            reviews_fb_1.append('')
            reviews_fb_2.append('')
            reviews_date_1.append('undefined')
            reviews_date_2.append('undefined')
        else:
            reviews_fb_1.append(str_review[0][0])
            reviews_date_1.append(datetime.strptime(str_review[1][0],'%m/%d/%Y')) 
            if len(str_review[0]) == 1:
                reviews_fb_2.append('')
                reviews_date_2.append('undefined')
            else:
                reviews_fb_2.append(str_review[0][1])
                reviews_date_2.append(datetime.strptime(str_review[1][1],'%m/%d/%Y'))

    df['reviews_fb_1'] = reviews_fb_1
    df['reviews_fb_2'] = reviews_fb_2
    df['reviews_date_1'] = reviews_date_1
    df['reviews_date_2'] = reviews_date_2


processing_reviews(df)

In [23]:
def get_difference(d1,d2):
    if type(d1) and type(d2) == datetime:
        return (d1-d2).days
    else:
        return 0

df['distance_between_reviews'] = df.apply(lambda x: get_difference(x['reviews_date_1'],x['reviews_date_2']),axis=1)

# сразу посмотрим

df['distance_between_reviews']

0          41
1         382
2           2
3           0
4         272
         ... 
39995      34
39996       9
39997    3127
39998      23
39999    1306
Name: distance_between_reviews, Length: 40000, dtype: int64

In [24]:
df.reviews_date_2

0        2017-11-20 00:00:00
1        2016-06-19 00:00:00
2        2018-01-06 00:00:00
3                  undefined
4        2017-02-19 00:00:00
                ...         
39995    2017-11-12 00:00:00
39996    2017-12-12 00:00:00
39997    2008-04-12 00:00:00
39998    2017-06-18 00:00:00
39999    2012-07-04 00:00:00
Name: reviews_date_2, Length: 40000, dtype: object

In [25]:
def get_prescription(d1,d2):
    if type(d1) and type(d2) == datetime:
        if d2 > d1:
            max_date = d2
        else:
            max_date = d1
        
        return (datetime.now() - max_date).days
    else:
        return 0

df['prescription'] = df.apply(lambda x: get_prescription(x['reviews_date_1'],x['reviews_date_2']),axis=1)

In [26]:
def last_review(d1,d2):
    if type(d1) and type(d2) == datetime:
        if d2 > d1:
            max_date = d2
        else:
            max_date = d1
        
        return max_date
df['prescription'] = df.apply(lambda x: last_review(x['reviews_date_1'],x['reviews_date_2']),axis=1)

In [27]:
df['prescription'].sort_values(ascending=False)

17447   2018-02-26
26781   2018-02-26
22880   2018-02-26
37689   2018-02-26
38749   2018-02-26
           ...    
39986          NaT
39988          NaT
39989          NaT
39991          NaT
39994          NaT
Name: prescription, Length: 40000, dtype: datetime64[ns]

In [28]:

def get_difference(d1,d2):
    if type(d1) and type(d2) == datetime:
        return (d1-d2).days
    else:
        return 0

df['distance_between_reviews'] = df.apply(lambda x: get_difference(x['reviews_date_1'],x['reviews_date_2']),axis=1)

# сразу посмотрим

df['distance_between_reviews']

0          41
1         382
2           2
3           0
4         272
         ... 
39995      34
39996       9
39997    3127
39998      23
39999    1306
Name: distance_between_reviews, Length: 40000, dtype: int64

In [29]:

df.info()
df['number_of_reviews'].fillna(0,inplace = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   restaurant_id             40000 non-null  object        
 1   city                      40000 non-null  object        
 2   cuisine_style             40000 non-null  object        
 3   ranking                   40000 non-null  float64       
 4   rating                    40000 non-null  float64       
 5   price_range               40000 non-null  int64         
 6   number_of_reviews         40000 non-null  float64       
 7   reviews                   40000 non-null  object        
 8   url_ta                    40000 non-null  object        
 9   id_ta                     40000 non-null  object        
 10  number_of_reviews_NaN     40000 non-null  bool          
 11  number_of_cuisines        40000 non-null  int64         
 12  reviews_fb_1      

In [30]:
Restaurants_in_City = df.city.value_counts()

In [31]:
df['Restaurants_in_City'] = df.city.apply(lambda x: Restaurants_in_City[x])

In [32]:
def restaurants_in_city(data):
    Restaurants_in_city = df.city.value_counts()
    df['Restaurants_in_city'] = df.city.apply(lambda x: Restaurants_in_city[x])
    return data

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   restaurant_id             40000 non-null  object        
 1   city                      40000 non-null  object        
 2   cuisine_style             40000 non-null  object        
 3   ranking                   40000 non-null  float64       
 4   rating                    40000 non-null  float64       
 5   price_range               40000 non-null  int64         
 6   number_of_reviews         40000 non-null  float64       
 7   reviews                   40000 non-null  object        
 8   url_ta                    40000 non-null  object        
 9   id_ta                     40000 non-null  object        
 10  number_of_reviews_NaN     40000 non-null  bool          
 11  number_of_cuisines        40000 non-null  int64         
 12  reviews_fb_1      

In [34]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. 
df = pd.get_dummies(df, columns=[ 'city',], dummy_na=True)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   restaurant_id             40000 non-null  object        
 1   cuisine_style             40000 non-null  object        
 2   ranking                   40000 non-null  float64       
 3   rating                    40000 non-null  float64       
 4   price_range               40000 non-null  int64         
 5   number_of_reviews         40000 non-null  float64       
 6   reviews                   40000 non-null  object        
 7   url_ta                    40000 non-null  object        
 8   id_ta                     40000 non-null  object        
 9   number_of_reviews_NaN     40000 non-null  bool          
 10  number_of_cuisines        40000 non-null  int64         
 11  reviews_fb_1              40000 non-null  object        
 12  reviews_fb_2      

In [36]:
df=df.drop([ 'restaurant_id','cuisine_style','reviews','url_ta','id_ta', 'reviews_fb_1', 'reviews_fb_2','reviews_date_1','reviews_date_2','prescription'], axis = 1)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ranking                   40000 non-null  float64
 1   rating                    40000 non-null  float64
 2   price_range               40000 non-null  int64  
 3   number_of_reviews         40000 non-null  float64
 4   number_of_reviews_NaN     40000 non-null  bool   
 5   number_of_cuisines        40000 non-null  int64  
 6   distance_between_reviews  40000 non-null  int64  
 7   Restaurants_in_City       40000 non-null  int64  
 8   city_Amsterdam            40000 non-null  uint8  
 9   city_Athens               40000 non-null  uint8  
 10  city_Barcelona            40000 non-null  uint8  
 11  city_Berlin               40000 non-null  uint8  
 12  city_Bratislava           40000 non-null  uint8  
 13  city_Brussels             40000 non-null  uint8  
 14  city_B

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [38]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['rating'], axis = 1)
y = df['rating']

In [39]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [40]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [41]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [42]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [43]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2089895
